# Expert Panel Agents Exercise

In this exercise, we'll create a panel of expert agents that:
1. Collaborate on complex problems
2. Reach consensus through discussion
3. Provide specialized insights

Focus on creating effective agent groups!

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Setup  

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="agent"))

### 2. Define Expert Instructions (Experiment with these!)

Try modifying these instructions to create different expert personalities!

In [ ]:
# Technical Expert Instructions
# Try adjusting the expertise areas or analysis approach!
technical_expert_instructions = """
You are a Technical Expert specializing in implementation and feasibility.

Expertise Areas:
- System architecture
- Technical constraints
- Implementation challenges

Analysis Approach:
1. Evaluate technical feasibility
2. Identify potential challenges
3. Suggest practical solutions

Communication Style:
- Clear and precise
- Evidence-based arguments
- Implementation-focused
"""

# Business Expert Instructions
# Try modifying the evaluation criteria or strategic focus!
business_expert_instructions = """
You are a Business Expert focusing on market viability and strategy.

Expertise Areas:
- Market analysis
- Business models
- ROI evaluation

Analysis Approach:
1. Assess market potential
2. Evaluate business impact
3. Consider competitive landscape

Communication Style:
- Strategic perspective
- Data-driven insights
- ROI-focused
"""

# User Experience Expert Instructions
# Try changing the user considerations or testing methods!
ux_expert_instructions = """
You are a User Experience Expert advocating for user needs and accessibility.

Expertise Areas:
- User behavior
- Interface design
- Accessibility standards

Analysis Approach:
1. Consider user impact
2. Evaluate usability
3. Suggest improvements

Communication Style:
- User-centered perspective
- Empathy-driven
- Solution-oriented
"""

# Create expert agents
technical_expert = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="TechnicalExpert",
    instructions=technical_expert_instructions
)

business_expert = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="BusinessExpert",
    instructions=business_expert_instructions
)

ux_expert = ChatCompletionAgent(
    service_id="agent",
    kernel=kernel,
    name="UXExpert",
    instructions=ux_expert_instructions
)

### 3. Create Consensus Strategy (Experiment with this!)

Try modifying how agents reach agreement!

In [ ]:
class ConsensusStrategy(TerminationStrategy):
    """Strategy for determining when experts have reached consensus."""
    
    def __init__(self, required_agreements=2, max_iterations=10):
        self.required_agreements = required_agreements
        self.max_iterations = max_iterations
        self.iteration = 0
    
    async def should_agent_terminate(self, agent, history) -> bool:
        """Check if consensus has been reached."""
        self.iteration += 1
        
        # Check max iterations
        if self.iteration >= self.max_iterations:
            return True
        
        # Look for agreement indicators in recent messages
        recent_messages = history[-3:] if len(history) > 3 else history
        agreements = sum(1 for msg in recent_messages if "agree" in msg.content.lower())
        
        return agreements >= self.required_agreements

# Create expert panel with consensus strategy
expert_panel = AgentGroupChat(
    agents=[technical_expert, business_expert, ux_expert],
    termination_strategy=ConsensusStrategy(
        required_agreements=2,
        max_iterations=10
    )
)

### 4. Example Discussion

Try these examples, then modify the expert instructions and consensus strategy to see how it changes their interaction!

In [ ]:
# Example 1: Product Feature Discussion
print("Discussing new product feature...\n")

question = "Should we implement a voice-controlled interface for our mobile app?"
await expert_panel.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=question))
print(f"Question: {question}\n")

async for response in expert_panel.invoke():
    print(f"{response.role} - {response.name}: {response.content}\n")

print(f"Discussion complete: {expert_panel.is_complete}\n")

# Example 2: Technology Decision
print("\nDiscussing technology choice...\n")

question = "Should we migrate our database to a cloud-based solution?"
await expert_panel.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=question))
print(f"Question: {question}\n")

async for response in expert_panel.invoke():
    print(f"{response.role} - {response.name}: {response.content}\n")

print(f"Discussion complete: {expert_panel.is_complete}\n")

### 5. Additional exercises

Try these exercises to experiment with expert panels:

1. Modify Expert Instructions to create:
   - Different specializations
   - Various expertise levels
   - Unique perspectives

2. Enhance the Consensus Strategy with:
   - Weighted agreements
   - Different termination conditions
   - Quality metrics

3. Create new expert types:
   - Security Expert
   - Data Privacy Expert
   - Compliance Expert

4. Modify group dynamics by:
   - Adding a moderator agent
   - Implementing voting systems
   - Creating debate formats

5. Advanced Challenge:
   - Create specialized panels for different domains
   - Implement hierarchical decision-making
   - Add conflict resolution mechanisms

Remember: Focus on how different expert perspectives and group dynamics affect decision-making!